<a href="https://colab.research.google.com/github/lamachina/IT-PROJECT-FH---Trading-Robot/blob/main/Sniper_listing_kucoin_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ccxt
import time

class SpotKucoin():
    def __init__(self, apiKey=None, secret=None, password=None):
        kucoinAuthObject = {
            "apiKey": apiKey,
            "secret": secret,
            "password": password,
        }
        if kucoinAuthObject['secret'] == None:
            self._auth = False
            self._session = ccxt.kucoin()
        else:
            self._auth = True
            self._session = ccxt.kucoin(kucoinAuthObject)
        self.market = self._session.load_markets()

    def reload_markets(self):
        self.market = self._session.load_markets()

    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._auth:
                print("You must be authenticated to use this method", fn)
                exit()
            else:
                return fn(self, *args, **kwargs)
        return wrapped

    def convert_amount_to_precision(self, symbol, amount):
        return self._session.amount_to_precision(symbol, amount)

    def convert_price_to_precision(self, symbol, price):
        return self._session.price_to_precision(symbol, price)

    @authentication_required
    def place_market_order(self, symbol, side, amount):
        try:
            return self._session.createOrder(
                symbol, 
                'market', 
                side, 
                self.convert_amount_to_precision(symbol, amount),
                None
            )
        except Exception as err:
            raise err

    @authentication_required
    def place_limit_order(self, symbol, side, amount, price):
        try:
            return self._session.createOrder(
                symbol, 
                'limit', 
                side, 
                self.convert_amount_to_precision(symbol, amount), 
                self.convert_price_to_precision(symbol, price)
                )
        except Exception as err:
            raise err

kucoin = SpotKucoin(
    apiKey="384b57ep8807o48a1113f3f7fc",
    secret="186354ba-9dep-4t5n-4644-bcbc78d611pr",
    password="motdepassedelapi"
)

symbol = "KICKS/USDT"
amount = 1000
sell_price = 0.09
# KICKS - Issue Price : $0.005 - 2022-10-18 12:00 UTC+2
# Amount: le nombre de coins que tu veux buy par transaction
# Sell price : le prix de vente

while True:
    try:
        kucoin.reload_markets()

        kucoin.place_market_order(symbol, "buy", amount)
        print("buy")

        kucoin.place_limit_order(symbol, "sell", amount, sell_price)
        print("sell")
        break
    except Exception as err:
        print(err)
        if str(err) == "kucoin does not have market symbol " + symbol:
            time.sleep(0.1)
        pass